In [3]:
import torch
import torchvision
import os
from torchvision import transforms
from os import path, mkdir
from PIL import Image, ImageOps
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader

convert_tensor = transforms.ToTensor()
images = 16 #Amount of images you want to generate. Can be modified
background = 125
char  = 3
count = 0
min_char_size = 50
max_char_size = 150
dataset = []
for i in range(images):
    bg_rand = np.random.randint(1, background + 1)
    bgString = '../data/background/nature ({}) (Copy).jpg'.format(bg_rand)
    char_rand = np.random.randint(0, char)
    size_rand = np.random.randint(min_char_size, max_char_size)
    if i%2 == 0:
        charString = '../data/waldo/char{}.png'.format(char_rand)
        label = torch.tensor([1, 0], dtype=torch.int32)
    else:
        charString = '../data/cat/notchar{}.png'.format(char_rand)
        label = torch.tensor([0, 1], dtype=torch.int32)
    bg = Image.open(bgString).convert("RGBA")
    bg_x, bg_y = bg.size
    im = Image.open(charString)
    im = im.resize((size_rand,size_rand))
    rand_pert = np.random.randint(0, 4)
    if rand_pert == 0:
        im = ImageOps.flip(im)
    elif rand_pert == 1:
        im = ImageOps.mirror(im)
    elif rand_pert == 2:
        im = ImageOps.flip(im)
        im = ImageOps.mirror(im)
    im_x, im_y = im.size
    rand_x = random.randint(0, bg_x-im_x)
    rand_y = random.randint(0, bg_y-im_y)
    bg.paste(im,(rand_x,rand_y),mask=im)
    bg_tensor = convert_tensor(bg)
    dataset.append((bg_tensor,label))
    #output_folder = 'test'
    #output_filename = 'test{}.png'.format(count)  # Specify the desired output file name with the .png extension
    #output_path = os.path.join(output_folder, output_filename)
    #if os.path.exists(output_path):
    #    os.chmod(output_path, 0o777)
    #bg.save(output_path, "PNG")
    count += 1
        
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, label = self.data[index]
        return image, label
    
# Convert your dataset list to a CustomDataset object
custom_dataset = CustomDataset(dataset)




In [6]:
batch_size = 4  # You can adjust this according to your needs
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False)
for images, labels in data_loader:
    print(labels)

tensor([[1, 0],
        [0, 1],
        [1, 0],
        [0, 1]], dtype=torch.int32)
tensor([[1, 0],
        [0, 1],
        [1, 0],
        [0, 1]], dtype=torch.int32)
tensor([[1, 0],
        [0, 1],
        [1, 0],
        [0, 1]], dtype=torch.int32)
tensor([[1, 0],
        [0, 1],
        [1, 0],
        [0, 1]], dtype=torch.int32)
